# Database overview notebook

> First time use: follow instructions in the README.md file in this directory.

In [8]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info()


Timelink version: 1.1.25
Project name: dehergne
Project home: /Users/jrc/mhk-home/sources/dehergne
Database type: sqlite
Database name: dehergne
Kleio image: timelinkserver/kleio-server
Kleio server token: Ww6W5...
Kleio server URL: http://127.0.0.1:8090
Kleio server home: /Users/jrc/mhk-home/sources/dehergne
Kleio server container: mystifying_beaver
Kleio version requested: latest
Kleio server version: 12.8.593 (2025-03-16 21:55:53)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne, project_home=/Users/jrc/mhk-home/sources/dehergne, db_type=sqlite, db_name=dehergne, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


## Ligar à base de dados

---

## Connect to database

In [9]:
from timelink.api.database import TimelinkDatabase

# to connect to a postgresql database
# do: db_type='postgresql' and db_name='timelink' and db_user='postgres' and db_password='postgres' and db_host='localhost'

db: TimelinkDatabase = tlnb.db

## Database status

In [10]:
tlnb.table_row_count_df()

,table,count
0,acts,29
1,alembic_version,1
2,aregisters,1
3,attributes,26742
4,blinks,200
5,class_attributes,70
6,classes,14
7,entities,32950
8,geoentities,359
9,goods,0


## Recent sources

In [12]:
number_of_sources = 100

Source = db.get_model('source')

with db.session() as session:
    sources = session.query(Source).order_by(Source.updated.desc()).all()
    for source in sources[:number_of_sources]:
        print(f"{source.id:32} {source.updated:%Y-%m-%d %H:%M:%S} {source.kleiofile}")


dehergne-z                       2025-05-10 03:53:41 /kleio-home/sources/dehergne-z.cli
dehergne-y                       2025-05-10 03:53:38 /kleio-home/sources/dehergne-y.cli
dehergne-x                       2025-05-10 03:53:33 /kleio-home/sources/dehergne-x.cli
dehergne-w                       2025-05-10 03:53:31 /kleio-home/sources/dehergne-w.cli
dehergne-v                       2025-05-10 03:53:20 /kleio-home/sources/dehergne-v.cli
dehergne-u                       2025-05-10 03:53:17 /kleio-home/sources/dehergne-u.cli
dehergne-t                       2025-05-10 03:53:06 /kleio-home/sources/dehergne-t.cli
dehergne-s                       2025-05-10 03:52:52 /kleio-home/sources/dehergne-s.cli
dehergne-r                       2025-05-10 03:52:40 /kleio-home/sources/dehergne-r.cli
dehergne-q                       2025-05-10 03:52:37 /kleio-home/sources/dehergne-q.cli
dehergne-p                       2025-05-10 03:52:30 /kleio-home/sources/dehergne-p.cli
dehergne-o                      

### Tipos de attributos

---

### Attribute types

In [17]:
import pandas as pd

from sqlalchemy import func
from sqlalchemy import select


pd.set_option('display.max_rows', 500)

attr_table = tlnb.db.get_table('attributes')
tlnb.db.describe('attributes', show=True)
print()
stmt = select(
    attr_table.c.the_type,
    func.count().label('count'),
    func.count(func.distinct(attr_table.c.the_value)).label('distinct_value')
    ).group_by('the_type')
print(stmt)
print()

with tlnb.db.session() as session:
    # nml2 = session.query(Attribute.the_type,func.count().label('tot')).group_by(Attribute.the_type).all()
    nml = session.execute(stmt)
    attribute_df = pd.DataFrame(nml)

attribute_df

attributes (model_table)
id                   entities             VARCHAR    
class                entities             VARCHAR    
inside               entities             VARCHAR    {ForeignKey('entities.id')}
the_source           entities             VARCHAR    
the_order            entities             INTEGER    
the_level            entities             INTEGER    
the_line             entities             INTEGER    
groupname            entities             VARCHAR    
extra_info           entities             JSON       
updated              entities             DATETIME   
indexed              entities             DATETIME   
id                   attributes           VARCHAR    {ForeignKey('entities.id')}
entity               attributes           VARCHAR    {ForeignKey('entities.id')}
the_type             attributes           VARCHAR    
the_value            attributes           VARCHAR    
the_date             attributes           VARCHAR    
obs                  attribute

,the_type,count,distinct_value
0,activa,286,3
1,alternative-name,4,4
2,alternative-name@wikidata,3,3
3,baptizado,28,28
4,baptizado@wikidata,26,26
5,bibliografia,9,9
6,cargo,356,225
7,chegada,487,71
8,chegada@wikidata,420,61
9,dehergne,1464,1211


## Vocabulary of attributes

In [16]:
from timelink.pandas import attribute_values

df_totals = attribute_values('jesuita-fundador',db=tlnb.db)
df_totals

,count,date_min,date_max
value,,,
Paris,3,15340815,15340815


## Mostrar uma fonte

---

## Show a source

In [7]:
id = 'dehergne-locations-1644'
with db.session() as session:
    source = session.get(Source,id)
    print(source.to_kleio())

fonte$dehergne-locations-1644/0/type=geoinformation/ref=None/loc=None/kleiofile="/kleio-home/sources/dehergne-locations-1644.cli""/replaces=None/obs="""
      Planche: Carte des Chrétientés Chinoises de la fin des Ming (1644).p. 353

      _Diocèse_
      Depuis 1514, l'évêché de Funchal (île de Madère)
      s'étend jusqu'aux extrémi-
      tés de l'Asie. Depuis le 3 nov. 1534 est
      créé l'évêché de Goa, dont est déta-
      ché, le 23 janv. 1576, le diocèse de
      Macao, qui sera amputé, 14 février
      1588, du Japon (évêché de Funai):
      l'évêché de Macao comprend théori-
      quement tout l'empire de Chine.

      _Provinces Jésuites_

      la vice-province du Japon (province en
      1611); en 1623, la vice-province de Chine
      indépendante du Japon en est déta-
      chée; mais un visiteur jésuite contrôle
      l'une et l'autre.

      La Chine impériale distingue dans chaque province civile les _fou_, préfec-
      tures ou villes de premier ordre, dont dépenden

## Obter modelos para acesso aos dados

---

## Obtain models for data access

In [8]:
db.get_models_ids()


['entity',
 'attribute',
 'relation',
 'act',
 'source',
 'aregister',
 'person',
 'object',
 'geoentity',
 'rentity',
 'class']

In [9]:
Model = db.get_model('geoentity')

with db.session() as session:
    entities = session.query(Model).all()
    for entity in entities[:10]:
        print(entity.to_kleio())
        print()

geo1$Chekiang#Tche-kiang, hoje:Zhejiang, 浙江, @wikidata:Q16967 @dehergne:396/geo1
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q16967"#Tche-kiang, hoje:Zhejiang, 浙江, @wikidata:Q16967 @dehergne:396%Q16967/1644
  atr$geoentity:name@dehergne/"https://archive.org/details/bhsi37/page/n396/mode/1up"#Tche-kiang, hoje:Zhejiang, 浙江, @wikidata:Q16967 @dehergne:396%396/1644

geo2$Hangchou#Hang-tcheou, hoje: Hangzhou, 杭州, @wikidata:Q4970/geo2
  atr$activa/sim/1611
  atr$residencia-missao/Jesuíta/1611
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q4970"#Hang-tcheou, hoje: Hangzhou, 杭州, @wikidata:Q4970%Q4970/1644

geo3$Fuyang#Fou-yang, hoje:Fuyang, 富阳, @wikidata:Q1011103/geo3
  atr$activa/sim/1642
  atr$geoentity:name@wikidata/"https://www.wikidata.org/wiki/Q1011103"#Fou-yang, hoje:Fuyang, 富阳, @wikidata:Q1011103%Q1011103/1644

geo3$Jenho#Jen-houo, hoje: Renhe, 仁和县, Historical county name. coordinate: 30.448897N, 120.307504E/geo3/obs=no wikidata.
  atr$activa/sim/1608

g